# Train you first forecasting model

So you picked the forecasting project. Good for you! Forecasting is an important skill, and it's helpful to learn about the techchnologies out there that can make your life easier. Or worse, depending on how you look at it. ;)

### 1. Download the Kaggle forecasting data
Use the notebook example for this.

### 2. Access your data

In [ ]:
import pandas as pd
import json
import sagemaker
import boto3

In [ ]:
df = pd.read_csv('train.csv')

In [ ]:
df.shape

In [ ]:
df.head()

Each row here is a new point in time, and each column is an energy station. That means that each COLUMN is a unique time series data set. We are going to train our first model on a single column. Then, you can extend it by adding more columns.

In [ ]:
target = df['ACME']

In [ ]:
target.shape

Great! We have 5113 obervations. That's well over the 300-limit on DeepAR.

### 3. Create Train and Test Sets
Now, we'll build 2 datasets. One for training, another for testing. Both need to be written to json files, then copied over to S3.

In [ ]:
def get_split(df, freq='D', split_type = 'train', cols_to_use = ['ACME']):
    rt_set = []
    
    # use 70% for training
    if split_type == 'train':
        lower_bound = 0
        upper_bound = round(df.shape[0] * .7)
        
    # use 15% for validation
    elif split_type == 'validation':
        lower_bound = round(df.shape[0] * .7)
        upper_bound = round(df.shape[0] * .85)
        
    # use 15% for test
    elif split_type == 'test':
        lower_bound = round(df.shape[0] * .85)
        upper_bound = df.shape[0]
            
    # loop through columns you want to use
    for h in list(df):
        if h in cols_to_use:
            
            target_column = df[h].values.tolist()[lower_bound:upper_bound]
            
            date_str = str(df.iloc[0]['Date'])
            
            year = date_str[0:4]
            month = date_str[4:6]
            date = date_str[7:]
                                                
            start_dataset = pd.Timestamp("{}-{}-{} 00:00:00".format(year, month, date, freq=freq))
                        
            # create a new json object for each column
            json_obj = {'start': str(start_dataset),
                       'target':target_column}
    
            rt_set.append(json_obj)
    
    return rt_set

In [ ]:
train_set = get_split(df, freq)
test_set = get_split(df, freq, split_type = 'test')

In [ ]:
def write_dicts_to_file(path, data):
    with open(path, 'wb') as fp:
        for d in data:
            fp.write(json.dumps(d).encode("utf-8"))
#             fp.write("\n".encode('utf-8'))

In [ ]:
write_dicts_to_file('train.json', train_set)
write_dicts_to_file('test.json', test_set)

In [ ]:
!aws s3 cp train.json s3://forecasting-do-not-delete/train/train.json
!aws s3 cp test.json s3://forecasting-do-not-delete/test/test.json

### 4. Run a SageMaker Training Job
Ok! If everything worked, we should be able to train a model in SageMaker straight away.

In [ ]:
sess = sagemaker.Session()
region = sess.boto_region_name
image = sagemaker.amazon.amazon_estimator.get_image_uri(region, "forecasting-deepar", "latest")
role = sagemaker.get_execution_role()   

In [ ]:
estimator = sagemaker.estimator.Estimator(
    sagemaker_session=sess,
    image_name=image,
    role=role,
    train_instance_count=1,
    train_instance_type='ml.c4.2xlarge',
    base_job_name='deepar-electricity-demo',
    output_path='s3://forecasting-do-not-delete/output'
)

In [ ]:
hyperparameters = {
    
    # frequency interval is once per day
    "time_freq": 'D',
    "epochs": "400",
    "early_stopping_patience": "40",
    "mini_batch_size": "64",
    "learning_rate": "5E-4",
    
    # let's use the last 30 days for context
    "context_length": str(30),
    
    # let's forecast for 30 days
    "prediction_length": str(30)
}

estimator.set_hyperparameters(**hyperparameters)

In [ ]:
data_channels = {
    "train": "s3://forecasting-do-not-delete/train/train.json",
    "test": "s3://forecasting-do-not-delete/test/test.json"
}

estimator.fit(inputs=data_channels, wait=True)

### 5. Run Inference
If you made it this far, congratulations! None of this is easy. For your next steps, please open up the example notebook under the SageMakerExamples:
- SageMakerExamples/Introduction To Amazon Algorithms/DeepAR-Electricity.

That will walk you through both how to add more timeseries to your model, and how to get inference results out of it.

### 6. Extend Your Solution
Now you're getting forecasts, how will you extend your solution? How good are your forecasts? What about getting forecasts for the other stations? Is your model cognizant of the weather?

Spend your remaining time growing your modeling solution to leverage additional datasets. Then, think through how you'd set this up to run in production.